# Estimación por mínimos cuadrados
_Nicolás Villegas Vargas, María Camila Vásquez Correa_

Modelación Experimental, 2019-1

### ARMA: Datos artificiales

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import warnings
warnings.simplefilter('ignore')
%matplotlib inline

Generamos datos con los siguientes parámetros

In [2]:
A = np.matrix([[.75,-0.25],[1, 0]])
D = np.matrix([[.65,.35],[0,0]])

In [3]:
# Simulacion de datos
p = 2
q = 2
nobs = 250
X = np.zeros((250,p))
# condicion inicial
X[0] = np.array([-0.5,-1.4])
# Ruido
e = np.random.normal(size=nobs)

In [4]:
for i in range(1, nobs):
    X[i] = X[i-1]*A.transpose() + np.array([e[i-1],e[i]])*D.transpose()

In [5]:
trace = go.Scatter(
    y = X.T[1],
    x = np.arange(nobs)
)

py.iplot([trace], filename='basic-line')

In [6]:
X = np.matrix(X)

### Ordinary Least Squares

### Estimación de parámetros

In [7]:
def ols(X, t):
    V = X[0].transpose()*X[1]
    G = X[0].transpose()*X[0]
    for i in range(1,t):
        V += X[i].transpose()*X[i+1]
        G += X[i].transpose()*X[i]
    return V.transpose()*np.linalg.inv(G)    

In [26]:
c1 = []
c2 = []
for t in range(5, nobs-1):
    c = ols(X, t)
    c1.append(c.A1[0])
    c2.append(c.A1[1])

In [27]:
a1 = [A.A1[0] for _ in range(len(c1))]
a2 = [A.A1[1] for _ in range(len(c2))]

In [28]:
trace = go.Scatter(
    y = a1,
    x = np.arange(nobs),
    name = 'Parametro a1')

trace1 = go.Scatter(
    y = c1,
    x = np.arange(nobs),
    name = 'Estimacion a1')

trace2 = go.Scatter(
    y = a2,
    x = np.arange(nobs),
    name = 'Parametro a2')

trace3 = go.Scatter(
    y = c2,
    x = np.arange(nobs),
    name = 'Estimacion a2')

layout = go.Layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickcolor='rgb(204, 204, 204)',
        tickwidth=2,
        ticklen=5,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    )
)

annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Estimación con mínimos cuadrados ordinarios',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))

layout['annotations'] = annotations

layout['annotations'] = annotations
traces = [trace, trace1, trace2, trace3]

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig)


## Instrumental Variables

In [11]:
def IVols(X,t):
    V = X[0].transpose()*X[2]
    G = X[2].transpose()*X[1]
    for i in range(1,t):
        V += X[i].transpose()*X[i+2]
        G += X[i+2].transpose()*X[i]
    return V.transpose()*np.linalg.inv(G)    

In [24]:
X = np.matrix(X)
c1 = []
c2 = []
for t in range(5, nobs-1):
    c = IVols(X, t)
    c1.append(c.A1[0])
    c2.append(c.A1[1])

In [25]:
trace = go.Scatter(
    y = a1,
    x = np.arange(nobs),
    name = 'Parametro a1')

trace1 = go.Scatter(
    y = c1,
    x = np.arange(nobs),
    name = 'Estimacion a1')

trace2 = go.Scatter(
    y = a2,
    x = np.arange(nobs),
    name = 'Parametro a2')

trace3 = go.Scatter(
    y = c2,
    x = np.arange(nobs),
    name = 'Estimacion a2')

layout = go.Layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickcolor='rgb(204, 204, 204)',
        tickwidth=2,
        ticklen=5,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    )
)

annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Estimación con variables instrumentales',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))

layout['annotations'] = annotations

layout['annotations'] = annotations
traces = [trace, trace1, trace2, trace3]

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig)


### Recursive Least Squares

In [14]:
V0 = X[0].transpose()*X[1]
G0 = X[0].transpose()*X[0]
R0 = np.linalg.inv(G0)
C0 = V0.T*R0

In [15]:
R1 = R0 - (R0*X[1].T*X[1]*R0)/(1+X[1]*R0*X[1].T)

In [16]:
X[0]

matrix([[-0.5, -1.4]])

In [17]:
C1 = C0 + (X[2].T - C0*X[1].T)*X[1]*R1

In [18]:
ols(X,3)

matrix([[-0.18184678, -0.17510694],
        [ 1.        ,  0.        ]])

In [19]:
C1

matrix([[-2.61810061,  0.3090503 ],
        [ 0.90906844, -0.45453422]])

In [20]:
def RLS(X, t):
    V = X[0].transpose()*X[1]
    G = X[0].transpose()*X[0]
    R = np.linalg.inv(G)
    C = V0.T*R0
    for i in range(1, t):
        R -= (R*X[i].T*X[i]*R)/(1+X[i]*R*X[i].T)
        C += (X[i+1].T - C*X[i].T)*X[i]*R
    return C

In [21]:
RLS(X,0)

matrix([[ 0., -1.],
        [ 0.,  0.]])

In [22]:
c1 = []
c2 = []
for t in range(50, nobs):
    c = RLS(X, t)
    c1.append(c.A1[0])
    c2.append(c.A1[1])

In [23]:
trace = go.Scatter(
    y = a1,
    x = np.arange(nobs),
    name = 'Parametro a1')

trace1 = go.Scatter(
    y = c1,
    x = np.arange(nobs),
    name = 'Estimacion a1')

trace2 = go.Scatter(
    y = a2,
    x = np.arange(nobs),
    name = 'Parametro a2')

trace3 = go.Scatter(
    y = c2,
    x = np.arange(nobs),
    name = 'Estimacion a2')

layout = go.Layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickcolor='rgb(204, 204, 204)',
        tickwidth=2,
        ticklen=5,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    )
)

annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Estimación con mínimos cuadrados recursivos',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))

layout['annotations'] = annotations

layout['annotations'] = annotations
traces = [trace, trace1, trace2, trace3]

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig)
